In [1]:
#/home/labuser/Documents/Level3/Day3/DataDemo/customers.parquet
#/home/labuser/Documents/Level3/Day3/DataDemo/bank_loans.parquet
#/home/labuser/Documents/Level3/Day3/DataDemo/branches.parquets
#/home/labuser/Documents/Level3/Day3/DataDemo/transactions.parquet

In [2]:
# Before AQE

from pyspark.sql import SparkSession
import time

# Initialize the Spark Session

spark = SparkSession.builder.appName("AQE_Example").config("spark.sql.adaptive.enabled", "false").getOrCreate()

In [3]:
# Load the Parquet files

loan_df = spark.read.parquet("/home/labuser/Documents/Level3/Day3/DataDemo/bank_loans.parquet")
branch_df = spark.read.parquet("/home/labuser/Documents/Level3/Day3/DataDemo/branches.parquets")

In [4]:
# Start Timer
start_time = time.time()


In [5]:
# Filtering on branch_id but without purning - reads unnecessary data 

result_df = loan_df.join(branch_df, "branch_id", "inner").filter("branch_name = 'New York'")

In [6]:
# Trigger Action to Measure Time
result_df.show()

+---------+--------+-----------+-----------+-------------+------+---------------+-----------+--------+-----+--------+
|branch_id| loan_id|customer_id|loan_amount|interest_rate|tenure|approval_status|branch_name|    city|state|zip_code|
+---------+--------+-----------+-----------+-------------+------+---------------+-----------+--------+-----+--------+
| 2bde6bb2|08e84e24|      77377|   12547.47|         12.8|    12|        Pending|   New York|New York|   NY|   62264|
| 2bde6bb2|50764cb3|      88695|    34477.8|         12.4|    12|       Rejected|   New York|New York|   NY|   62264|
| 2bde6bb2|72315660|      57946|    4753.65|         9.75|    60|        Pending|   New York|New York|   NY|   62264|
| 2bde6bb2|6c30b86f|      25340|   39568.18|          3.5|    12|       Rejected|   New York|New York|   NY|   62264|
| 2bde6bb2|75bc8cca|      39480|    3060.82|         3.53|    24|        Pending|   New York|New York|   NY|   62264|
| 2bde6bb2|101e8743|      34773|   35502.34|         6.2

In [7]:
# End Timerend
end_time = time.time()

In [8]:
# Calculate Execution time 

execution_time = end_time - start_time
print(f"Execution Time : {execution_time:.2f} seconds")

Execution Time : 2.54 seconds


In [9]:
spark.conf.set("spark.sql.adaptive.enabled", "true")

# Start Timer
start_time = time.time()
#Optimized Query Executuion with AQE
result_df = loan_df.join(branch_df, "branch_id", "inner").filter("branch_name = 'New York'")

#Trigger an Action 

result_df.show()

# End Timerend
end_time = time.time()

# Calculate Execution time 

execution_time = end_time - start_time
print(f"Execution Time : {execution_time:.2f} seconds")

+---------+--------+-----------+-----------+-------------+------+---------------+-----------+--------+-----+--------+
|branch_id| loan_id|customer_id|loan_amount|interest_rate|tenure|approval_status|branch_name|    city|state|zip_code|
+---------+--------+-----------+-----------+-------------+------+---------------+-----------+--------+-----+--------+
| 2bde6bb2|08e84e24|      77377|   12547.47|         12.8|    12|        Pending|   New York|New York|   NY|   62264|
| 2bde6bb2|50764cb3|      88695|    34477.8|         12.4|    12|       Rejected|   New York|New York|   NY|   62264|
| 2bde6bb2|72315660|      57946|    4753.65|         9.75|    60|        Pending|   New York|New York|   NY|   62264|
| 2bde6bb2|6c30b86f|      25340|   39568.18|          3.5|    12|       Rejected|   New York|New York|   NY|   62264|
| 2bde6bb2|75bc8cca|      39480|    3060.82|         3.53|    24|        Pending|   New York|New York|   NY|   62264|
| 2bde6bb2|101e8743|      34773|   35502.34|         6.2